In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

# 1. Cargar y Preprocesar Datos (código que proporcionaste)
# Asegúrate de que la ruta al archivo CSV sea correcta.
try:
    # Reemplaza "path/to/your/Proyecto 2/FLIR_groups1and2.csv" con la ruta correcta
    file_path = "archivos/FLIR_groups1and2.csv" # Asumiendo que el archivo está en el mismo directorio o proporciona la ruta completa
    df = (pd.read_csv(file_path, sep=";" , header=2))[['Max1R13_1', 'Max1R13_2', 'Max1R13_3', 'Max1R13_4',
                                                      'Max1L13_1', 'Max1L13_2', 'Max1L13_3', 'Max1L13_4',
                                                      'T_Max1', 'T_Max2', 'T_Max3', 'T_Max4',
                                                      'T_FHCC1', 'T_FHCC2', 'T_FHCC3', 'T_FHCC4',
                                                      'aveOralM', 'Gender', 'Age', 'Ethnicity', 'T_atm', 'Humidity', 'Cosmetics','Distance']].copy()

    df['Max1R13'] = df[['Max1R13_1', 'Max1R13_2', 'Max1R13_3', 'Max1R13_4']].mean(axis=1, skipna=True).astype(float)
    df.drop(columns=['Max1R13_1', 'Max1R13_2', 'Max1R13_3', 'Max1R13_4'],inplace=True)

    df['Max1L13'] = df[['Max1L13_1', 'Max1L13_2', 'Max1L13_3', 'Max1L13_4']].mean(axis=1, skipna=True).astype(float)
    df.drop(columns=['Max1L13_1', 'Max1L13_2', 'Max1L13_3', 'Max1L13_4'],inplace=True)

    df['T_Max'] = df[['T_Max1', 'T_Max2', 'T_Max3', 'T_Max4']].mean(axis=1, skipna=True).astype(float)
    df.drop(columns=['T_Max1', 'T_Max2', 'T_Max3', 'T_Max4'],inplace=True)

    df['TF_HCC'] = df[['T_FHCC1', 'T_FHCC2', 'T_FHCC3', 'T_FHCC4']].mean(axis=1, skipna=True).astype(float)
    df.drop(columns=['T_FHCC1', 'T_FHCC2', 'T_FHCC3', 'T_FHCC4'],inplace=True)

    print("Columnas disponibles después del preprocesamiento:")
    print(df.columns)
    print("\nPrimeras filas del DataFrame procesado:")
    print(df.head())
    print("\nInformación del DataFrame procesado:")
    df.info()

    # 2. Seleccionar Variables
    # Asumiremos que 'aveOralM' es la variable dependiente (objetivo).
    # Seleccionaremos algunas variables numéricas y una categórica ('Gender') como predictores.
    # Puedes ajustar esta selección según los requisitos de tu proyecto.
    # Columnas candidatas: 'Age', 'T_atm', 'Humidity', 'Distance', 'Max1R13', 'Max1L13', 'T_Max', 'TF_HCC', 'Gender'
    

    dependent_var = 'aveOralM'
    # Empezaremos con un conjunto de predictores. Puedes expandirlo.
    independent_vars_numerical = ['Max1R13','Max1L13', 'T_Max', 'TF_HCC']

    # Filtrar solo las columnas numéricas que existen en el df
    independent_vars_numerical = [col for col in independent_vars_numerical if col in df.columns]

    selected_columns = [dependent_var] + independent_vars_numerical
    
    # Crear una copia para el modelado para no alterar el df original innecesariamente
    model_df = df[selected_columns].copy()

    # 3. Manejar Datos Faltantes (en las columnas seleccionadas para el modelo)
    # Primero, verificamos si 'aveOralM' tiene NaNs y los eliminamos, ya que es nuestro objetivo.
    model_df.dropna(subset=[dependent_var], inplace=True)
    # Para los predictores, podemos eliminar filas con NaNs o imputar. Aquí los eliminaremos por simplicidad.
    # Antes de eliminar, es bueno saber cuántos NaNs hay
    print(f"\nNaNs antes de eliminarlos de model_df (columnas seleccionadas):")
    print(model_df.isnull().sum())
    model_df.dropna(inplace=True)
    print(f"\nForma del DataFrame después de eliminar NaNs: {model_df.shape}")

    if model_df.empty:
        print("\nEl DataFrame está vacío después de eliminar NaNs. Verifica tus datos y el manejo de NaNs.")
        # Detener la ejecución si el DataFrame está vacío
        exit()


    # Definir X (predictores) e y (objetivo)
    X = model_df.drop(columns=[dependent_var])
    y = model_df[dependent_var]

    # 5. Dividir los Datos
    # test_size es la proporción del dataset a incluir en el split de test (ej. 0.2 = 20%)
    # random_state asegura que la división sea la misma cada vez que corras el código (reproducibilidad)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print(f"\nForma de X_train: {X_train.shape}, X_test: {X_test.shape}")

    # 6. Entrenar el Modelo (usando scikit-learn)
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)

    # 7. Evaluar el Modelo
    y_pred_train = lr_model.predict(X_train)
    y_pred_test = lr_model.predict(X_test)

    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)

    print(f"\n--- Evaluación del Modelo (Scikit-learn) ---")
    print(f"Coeficientes: {lr_model.coef_}")
    print(f"Intercepto: {lr_model.intercept_}")
    print(f"MSE (entrenamiento): {mse_train:.4f}")
    print(f"R² (entrenamiento): {r2_train:.4f}")
    print(f"MSE (prueba): {mse_test:.4f}")
    print(f"R² (prueba): {r2_test:.4f}")

    # 8. Interpretar el Modelo (usando statsmodels para un resumen más detallado)
    # statsmodels necesita que se añada una constante para el término del intercepto
    X_train_sm = sm.add_constant(X_train)
    X_test_sm = sm.add_constant(X_test) # Para consistencia, aunque no siempre se usa para predecir directamente con OLS

    ols_model = sm.OLS(y_train, X_train_sm)
    ols_results = ols_model.fit()

    print(f"\n--- Resumen del Modelo (Statsmodels OLS) ---")
    print(ols_results.summary())
    
    # Guardar el dataframe procesado y listo para el modelado (opcional)
    # model_df.to_csv("datos_procesados_para_regresion.csv", index=False)
    # print("\nDataFrame procesado guardado en 'datos_procesados_para_regresion.csv'")


except FileNotFoundError:
    print(f"Error: El archivo CSV no se encontró en la ruta especificada: {file_path}")
    print("Por favor, asegúrate de que el archivo 'FLIR_groups1and2.csv' esté en el directorio correcto o proporciona la ruta completa.")
except Exception as e:
    print(f"Ocurrió un error durante la ejecución: {e}")

Columnas disponibles después del preprocesamiento:
Index(['aveOralM', 'Gender', 'Age', 'Ethnicity', 'T_atm', 'Humidity',
       'Cosmetics', 'Distance', 'Max1R13', 'Max1L13', 'T_Max', 'TF_HCC'],
      dtype='object')

Primeras filas del DataFrame procesado:
   aveOralM  Gender    Age                  Ethnicity  T_atm  Humidity  \
0     36.59    Male  41-50                      White   24.0      28.0   
1     37.19  Female  31-40  Black or African-American   24.0      26.0   
2     37.34  Female  21-30                      White   24.0      26.0   
3     37.09  Female  21-30  Black or African-American   24.0      27.0   
4     37.04    Male  18-20                      White   24.0      27.0   

   Cosmetics  Distance  Max1R13  Max1L13    T_Max   TF_HCC  
0        NaN       0.8  35.0300  35.3775  35.6925  33.5775  
1        NaN       0.8  34.5500  34.5200  35.1750  34.0325  
2        NaN       0.8  35.6525  35.5175  35.9125  34.9000  
3        NaN       0.8  35.2225  35.6125  35.7200  34